In [ ]:
# Requirements Installations --- PYTHON 3.9.* REQUIRED
%pip install "git+https://github.com/tournesol-app/tournesol.git@solidago-pipeline#egg=solidago&subdirectory=solidago"
%pip install requests

In [ ]:
# Imports
import time
import math
import random
import requests
import datetime
import itertools
import networkx as nx
import dateutil.parser as dateparse
from networkx import NetworkXNoPath
from solidago.pipeline.inputs import TournesolInputFromPublicDataset

VData=dict # Type

# Parameters

JWT: Get it from tournesol.app

- open website, open dev tools, get any request to Tournesol api, see Request Headers, get `Authentication="Bearer ..."` value
- DO NOT SHARE THIS TOKEN TO ANYONE. NEVER. IN ANY CONDITIONS. Even support will never need it.
- This token expires after some time of inactivity. If tool fails, try to update the token first.

LNGS: Pick all the languages of the videos to keep

In [ ]:
# PARAMETERS
USERNAME='NatNgs'
JWT='Bearer xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
LNGS=['fr', 'en'] # Consolidation phase will only suggest videos in these languages


# Technical

Below is some technical stuff used for the suggestion mechanism.

Play all the next cells one by one, you do not need to change anything.

In [ ]:
PUBLIC_DATASET = TournesolInputFromPublicDataset.download()
USER_DATASET:list[list[str]] = (
	PUBLIC_DATASET.comparisons[PUBLIC_DATASET.comparisons.public_username == USERNAME]
	              .loc[PUBLIC_DATASET.comparisons.criteria == 'largely_recommended']
	              [['entity_a', 'entity_b']]
	              .merge(PUBLIC_DATASET.entity_id_to_video_id, left_on='entity_a', right_index=True)
	              .drop('entity_a', axis=1).rename(columns={'video_id': 'entity_a'})
	              .merge(PUBLIC_DATASET.entity_id_to_video_id, left_on='entity_b', right_index=True)
	              .drop('entity_b', axis=1).rename(columns={'video_id': 'entity_b'})
).values
print(f"Loaded {len(USER_DATASET)} public comparisons")

In [ ]:
# Functions utils
LAST_TNSL_CALL=datetime.datetime.now(datetime.timezone.utc)
def callTournesol(path: str):
	global LAST_TNSL_CALL
	BASE_URL='https://api.tournesol.app/'
	wait=1-(datetime.datetime.now(datetime.timezone.utc)-LAST_TNSL_CALL).total_seconds()
	if wait > 0:
		time.sleep(wait)
	response = requests.get(BASE_URL + path, headers={
		'Authorization': JWT
	})
	LAST_TNSL_CALL = datetime.datetime.now(datetime.timezone.utc)
	return response.json()

def callTournesolMulti(path: str, args:str=None, start:int=0, end:int=0) -> list[VData]:
	LIMIT=1000
	URL=f'{path}?limit={LIMIT}' + (('&' + args) if args else '')
	offset=start
	rs = callTournesol(URL + f'&offset={offset}')
	if not 'count' in rs or not 'results' in rs:
		print('##### ERROR #####')
		print(URL)
		print(rs)
		exit(-1)
		return []

	total=rs['count']
	allRes:list[VData] = rs['results']
	print(f'{len(allRes)}/{total}', end=' ')

	while len(allRes) < total and (end <= 0 or offset < end):
		offset += LIMIT
		rs = callTournesol(URL + f'&offset={offset}')
		total=rs['count']
		allRes += rs['results']
		print(f'{len(allRes)}', end=' ')
	print()

	return allRes

def get(json:VData, default, *fields):
	for f in fields:
		if f in json:
			json = json[f]
			if json is None:
				return default
		else:
			return default
	return json

def rndAB():
	return ('A', 'B') if random.random() > 0.5 else ('B', 'A')

def get_individual_score(vdata:VData) -> float:
	arr = [s for s in get(vdata, [], 'individual_rating', 'criteria_scores') if s['criteria'] == 'largely_recommended']
	return arr[0]['score'] if arr else None

class IgnoreList:
	def __init__(self):
		self.vids:set[str] = set()
		self.channels:set[str] = set()
	
	def isIgnored(self, vdata:VData):
		if vdata['entity']['uid'] in self.vids:
			return True
		if vdata['entity']['metadata']['uploader'] in self.channels:
			return True
		return False
	
	def ignoreVideo(self, vid):
		self.vids.add(vid)
	def ignoreChannel(self, channel):
		self.channels.add(channel)

	def removeIgnored(self, vids:list[VData]):
		i=0
		while i < len(vids):
			if self.isIgnored(vids[i]):
				vids.pop(i)
			else:
				i+=1

IGNORE_LIST=IgnoreList()

In [ ]:
# Import video lists

def crossmerge_lists(l1, l2): 
	minln = min(len(l1), len(l2))
	merged = [None]*(minln*2)
	merged[::2] = l2[:minln]
	merged[1::2] = l1[:minln]
	merged += l2[minln:] + l1[minln:]
	return merged

def get_lists() -> tuple[list[VData], list[VData], list[VData]]:
	# Get
	print('Extracting compared videos...', end=' ')
	allRes:list[VData] = callTournesolMulti('users/me/contributor_ratings/videos', 'order_by=last_compared_at')

	print('Extracting rate_later list...', end=' ')
	allRes += callTournesolMulti('users/me/rate_later/videos')

	# Exclusions
	# Exclude videos when language not in LNGS & if has been compared within last 24h
	now = datetime.datetime.now()
	allRes = [v for v in allRes if
		get(v, LNGS[0], 'entity', 'metadata', 'language') in LNGS
		and (now - dateparse.parse(get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'), ignoretz=True)).total_seconds() > (60*60*24)
	]

	# Collect
	cmp_vids:list[dict[str,VData]] = [dict(), dict(), dict(), dict()] # min 4 to not explode
	for v in allRes:
		cmps = get(v, 0, 'individual_rating', 'n_comparisons')
		while cmps >= len(cmp_vids):
			cmp_vids.append(dict())
		cmp_vids[cmps][v['entity']['uid']] = v

	for i in range(len(cmp_vids)):
		print(f"\t- {i} cmps:", len(cmp_vids[i]))

	# rate_later
	v_0 = sorted(cmp_vids[0].values(), key=lambda v:(
		#	Well noted from few contributors => Go first
		get(v, 0, 'collective_rating', 'tournesol_score') / (get(v, None, 'collective_rating', 'n_contributors') if get(v, 0, 'collective_rating', 'n_contributors') > 1 else 2),
		#	When was added to rate later: recent first
		get(v, '9999-12-31T23:59:59', 'rate_later_metadata', 'created_at'),
		#	Publication date: recent first
		get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	), reverse=True)
	v_1 = sorted(cmp_vids[1].values(), key=lambda v:(
		#	Well noted from few contributors => Go first
		-get(v, 0, 'collective_rating', 'tournesol_score') / (get(v, None, 'collective_rating', 'n_contributors') if get(v, 0, 'collective_rating', 'n_contributors') > 1 else 2),
		#	Oldest last comparison first
		 get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		 get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))
	# Put first from v_1 first, then alternate between v_0 & v_1 in order until one of them is exhausted, then push the rest
	rate_later = crossmerge_lists(v_1, v_0)

	# low_comparisons
	v_2 = sorted(cmp_vids[2].values(), key=lambda v:(
		#	oldest last comparison first
		get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))
	v_3 = sorted(cmp_vids[3].values(), key=lambda v:(
		#	oldest last comparison first
		get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))
	low_comparisons = crossmerge_lists(v_2, v_3)

	# well_compared
	v_4_p = itertools.chain.from_iterable(cmp_vids[i].values() for i in range(4, len(cmp_vids)))
	well_compared = sorted(v_4_p, key=lambda v:(
		#	oldest last comparison first
		get(v, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at'),
		#	Publication date: older first
		get(v, '2000-01-01T00:00:00', 'entity', 'metadata', 'publication_date'),
	))

	return (rate_later, low_comparisons, well_compared)

In [ ]:
# Cache DistComparisonare_sharing_comparisonser
class DistComparisonChecker:
	def __init__(self, comparisons:list[list[str]]=list):
		self.graph = nx.Graph()
		for pair in comparisons:
			vid1 = 'yt:' + pair[0]
			vid2 = 'yt:' + pair[1]
			self.graph.add_edge(vid1, vid2)
		
		self.toupdate: set[str] = set(self.graph.nodes)

	def _update_cache(self, vid:str):
		if vid in self.toupdate or vid not in self.graph:
			print(f'Obtaining comparisons with {vid}...', end=' ')
			allRes = callTournesolMulti(f"users/me/comparisons/videos/{vid}/")
			self.graph.add_node(vid)
			for g in allRes:
				vid2 = g['entity_b' if g['entity_a']['uid'] == vid else 'entity_a']['uid']
				if not vid2 in self.graph:
					self.toupdate.add(vid2)
				self.graph.add_edge(vid, vid2)
			if vid in self.toupdate:
				self.toupdate.remove(vid)

	def update_cache_neighboors(self, vdata:VData):
		self._update_cache(vdata['entity']['uid'])
		for v in self.graph.neighbors(vdata['entity']['uid']):
			self._update_cache(v)

	def are_sharing_comparisons(self, vdata1:VData, vdata2:VData, cached:bool = False) -> bool:
		vid1 = vdata1['entity']['uid']
		vid2 = vdata2['entity']['uid']
		if not cached:
			self._update_cache(vid1)
			self._update_cache(vid2)
		if vid1 not in self.graph or vid2 not in self.graph:
			return False
		c1 = {v for v in self.graph[vid1]}
		c2 = {v for v in self.graph[vid2]}
		return (vid1 in c2) or (vid2 in c1) or (not c1.isdisjoint(c2))

	def addAsCompared(self, vdata1:VData, vdata2:VData):
		vid1 = vdata1['entity']['uid']
		vid2 = vdata2['entity']['uid']

		if not self.graph.has_edge(vid1, vid2):
			self._update_cache(vid1)
			self._update_cache(vid2)
			# Update cache and then retries
		
		if not self.graph.has_edge(vid1, vid2):
			self.graph.add_edge(vid1, vid2)
			self.toupdate.add(vid1)
			self.toupdate.add(vid2)

			if not vdata1.get('individual_rating', None):
				vdata1['individual_rating'] = dict()
			vdata1['individual_rating']['last_compared_at'] = datetime.datetime.now(datetime.timezone.utc).isoformat()
			vdata1['individual_rating']['n_comparisons'] = vdata1['individual_rating'].get('n_comparisons',0) + 1

			if not vdata2.get('individual_rating', None):
				vdata2['individual_rating'] = dict()
			vdata2['individual_rating']['last_compared_at'] = datetime.datetime.now(datetime.timezone.utc).isoformat()
			vdata2['individual_rating']['n_comparisons'] = vdata2['individual_rating'].get('n_comparisons',0) + 1

DCC = DistComparisonChecker(USER_DATASET)

In [ ]:
# Phase Init (Create first comparison from rate_later & low_comparisons)
def phase_init(rate_later:list[VData], low_comparisons:list[VData], DCC: DistComparisonChecker) -> tuple[VData, str]:
	"""
	Get the latest compared video in low_comparisons, and compare it to next rate_later video
	"""
	vid_new = rate_later.pop(0)

	vid_old:VData = None
	while not vid_old or DCC.are_sharing_comparisons(vid_new, vid_old):
		vid_old = max(
			[vdata for vdata in low_comparisons if not DCC.are_sharing_comparisons(vid_new, vdata, cached=True)],
			key=lambda vdata:(
				-get(vdata, 0, 'individual_rating', 'n_comparisons'), # Fewer comparisons first
				get(vdata, '2000-01-01T00:00:00', 'individual_rating', 'last_compared_at') # Compared the more recently
				# Publication date older first (low_comparisons list order)
			)
		)
	low_comparisons.remove(vid_old)

	DCC.addAsCompared(vid_new, vid_old)
	ab = rndAB()
	return (vid_new, f'[1] https://tournesol.app/comparison?uid{ab[0]}=' + vid_new['entity']['uid'] + f"&uid{ab[1]}=" + vid_old['entity']['uid'])


In [ ]:
# Phase Expand (Create next comparison from rate_later)
def phase_expand(vid_old:VData, rate_later:list[VData], DCC: DistComparisonChecker) -> tuple[VData, str]:
	"""
	Take first 2 in RATE_LATER

	If ko, change the 2nd one with the next and retry. If no more next: END
	When ok, print comparison URL, pop the first one & push it to the end of COMPARED, then go to Phase 1
	"""
	i = 0
	while DCC.are_sharing_comparisons(vid_old, rate_later[i]):
		i+=1
		if i > len(rate_later):
			# Pick any if all have shared comparisons
			i=0
			break

	vid_new = rate_later.pop(i)

	DCC.addAsCompared(vid_new, vid_old)
	ab = rndAB()
	return (vid_new, f'[+] https://tournesol.app/comparison?uid{ab[0]}=' + vid_new['entity']['uid'] + f"&uid{ab[1]}=" + vid_old['entity']['uid'])


In [ ]:
# Phase Intricate (Create next comparison from low_comparisons)
def phase_intricate(vid_new:VData, compared:list[VData], DCC: DistComparisonChecker) -> tuple[VData, str]:
	"""
	Take first from rate later
	Take first from compared

	Check for both: https://api.tournesol.app/users/me/comparisons/videos/yt:<vid>/
	There should be NO vid in common in both lists of entities.

	If ko, take next one from Compared and retry. If no more next: END
	When ok, print comparison URL and pop the one from COMPARED (if still less than 4 cmps, push it to the end of COMPARED), then go Phase 2
	"""
	i=0
	while DCC.are_sharing_comparisons(vid_new, compared[i]):
		i += 1
		if i > len(compared):
			# Pick any if all have shared comparisons
			i = 0
			break

	vid_old = compared.pop(i)
	DCC.addAsCompared(vid_new, vid_old)
	ab = rndAB()
	return (vid_new, f'[x] https://tournesol.app/comparison?uid{ab[0]}=' + vid_old['entity']['uid'] + f"&uid{ab[1]}=" + vid_new['entity']['uid'])


In [ ]:
# Phase Consolidate (Create comparison for 2 high_comparisons videos)
def phase_consolidate(high_comparisons:list[VData], DCC: DistComparisonChecker) -> str:
	"""
	From all compared videos, take only the ones that has:
		- been compared at least 3 time by me
		- been compared by at least 2 different contributors
		- not been compared in the last 6 days
	From these videos, get pairs such as videos in a pair has:
		- Same language
		- Exact same number of (individual) comparison made
		- No comparison in common (DCC)
	From all pairs, take the one having the smallest score, score being calculated by adding:
		+ Difference of individual score
		+ Difference of collective score
		+ Sqrt(Difference of duration)
		+ Sqrt(Difference of time between both video aired)
	
	Compare them (Do not remove them from the list)

	If none found, return None
	"""
	# Compute minmax & fast access to some data in vdata
	mins:dict[str,float] = dict()
	maxs:dict[str,float] = dict()
	vdata:list[dict[str,VData|float]] = []
	today = datetime.datetime.now()
	for v in high_comparisons:
		lng:str = get(v, None, 'entity', 'metadata', 'language')
		cmp:float = get(v, 0, 'individual_rating', 'n_comparisons')
		cnt:float = get(v, 0, 'collective_rating', 'n_contributors')
		indiv_score:float|None = get_individual_score(v)
		coll_score:float = get(v, 0, 'collective_rating', 'tournesol_score')
		duration:float = math.sqrt(get(v, 0, 'entity', 'metadata', 'duration'))
		aired:float = math.sqrt( (today - dateparse.parse(get(v, None, 'entity', 'metadata', 'publication_date'), ignoretz=True)).total_seconds() / (60*60*24) )
		last_cmp:float = (today - dateparse.parse(get(v, None, 'individual_rating', 'last_compared_at'), ignoretz=True)).total_seconds() / (60*60*24)
		
		if indiv_score is not None and cnt >= 2 and last_cmp > 6 and lng is not None:
			vdata.append({
				'#id': v['entity']['uid'],
				'lng': lng,
				'cmp': cmp,
				'ind': indiv_score,
				'col': coll_score,
				'dur': duration,
				'air': aired,
				'pub': get(v, True, 'individual_rating', 'is_public'),
				'full': v
			})
			for (key,val) in (('ind', indiv_score), ('col', coll_score), ('dur', duration), ('air', aired), ('cmp', cmp)):
				if not key in mins:
					mins[key] = val
					maxs[key] = val
				elif val < mins[key]:
					mins[key] = val
				elif val > maxs[key]:
					maxs[key] = val

	# Distance between nodes
	shortest_paths:dict[str,dict[str,int]] = dict(nx.all_pairs_shortest_path_length(DCC.graph))
	mins['pth'] = 3 # Min possible shorted path lenght for 2 nodes having no common comparison
	maxs['pth'] = max(d for bd in shortest_paths.values() for d in bd.values()) + 1 # If nodes are not connected, distance is "longuest path" +1

	mins['pub'] = 0
	maxs['pub'] = 2 # 2 if both are public, 1 if none are public, 0 if one is public but not the other

	coefs = { # + => Higher value is better, - => Lower value is better
		'ind': -2.0, # Prefer when there is not a lot of difference in individual score
		'dur': -1.0, # Prefer when there is not a lot of difference in video duration
		'pth': +0.5, # Prefer longer comparison chain length
		'cmp': -0.5, # Prefer when there is not a lot of comparisons
		'pub': +0.3, # Prefer when both video are public
		'air': -0.3, # Prefer when there is not a long time between both video aired
	}

	# Find best pair
	bestpair:tuple[dict[str,VData|float],dict[str,VData|float]]|None = None
	while bestpair is None:
		bestfitness:float = -999999
		for i1 in range(1,len(vdata)):
			v1 = vdata[i1]

			for v2 in vdata[0:i1]:
				if (v1['lng'] != v2['lng']
					or v1['cmp'] != v2['cmp']
					or DCC.are_sharing_comparisons(v1['full'], v2['full'], cached=True)
				):
					continue

				# shortest path
				distance = maxs['pth']
				try:
					distance = nx.shortest_path_length(DCC.graph, v1['#id'], v2['#id'])
				except:
					pass

				# Get pair score
				diffs:dict[str,float] = {
					'ind': abs(v1['ind']-v2['ind']),
					'dur': abs(v1['dur']-v2['dur']),
					'air': abs(v1['air']-v2['air']),
					'pth': distance-mins['pth'],
					'cmp': v1['cmp']-mins['cmp'],
					'pub': 2 if v1['pub'] and v2['pub'] else 0 if v1['pub'] or v2['pub'] else 1,
				}
				vals = {k: diffs[k]/(maxs[k]-mins[k]) for k in coefs}
				pcts = {k: v if coefs[k] > 0 else (1-v) for k,v in vals.items()}
				fitness = sum( abs(coefs[k]) * pcts[k] for k in coefs )
				if fitness > bestfitness:
					toprint = (
						  f"ind:{pcts['ind']:.0%}({v1['ind']:.0f}/{v2['ind']:.0f}), "
						+ f"dur:{pcts['dur']:.0%}, "
						+ f"air:{pcts['air']:.0%}, "
						+ f"pth:{pcts['pth']:.0%}({distance if distance < maxs['pth'] else '∞'}), "
						+ f"cmp:{pcts['cmp']:.0%}({v1['cmp']:.0f}), "
						+ f"pub:{pcts['pub']:.0%}({'Public' if v1['pub'] else 'Private'}/{'Public' if v2['pub'] else 'Private'})"
					)
					bestpair = (v1, v2, toprint, distance)
					bestfitness = fitness

		if bestpair is None:
			return None

		DCC.update_cache_neighboors(bestpair[0]['full'])
		DCC.update_cache_neighboors(bestpair[1]['full'])
		if DCC.are_sharing_comparisons(bestpair[0]['full'], bestpair[1]['full']):
			print('! Shared comparisons found')
			bestpair = None


		try:
			if nx.shortest_path_length(DCC.graph, bestpair[0]['#id'], bestpair[1]['#id']) < bestpair[3]:
				print('! Shorter path found')
				bestpair = None
		except:
			pass

	# print path between a and b
	try:
		print(' > '.join(nx.shortest_path(DCC.graph, bestpair[0]['#id'], bestpair[1]['#id'])))
	except:
		print('No path between', bestpair[0]['#id'], 'and', bestpair[1]['#id'])

	DCC.addAsCompared(bestpair[0]['full'], bestpair[1]['full'])
	maxfitness:float = sum(abs(c) for c in coefs.values())

	ab = rndAB()
	return (f'[*] https://tournesol.app/comparison?uid{ab[0]}=' + bestpair[0]['#id'] + f"&uid{ab[1]}=" + bestpair[1]['#id'] 
		 + f" [f={bestfitness/maxfitness:.2%}] ({bestpair[2]})"
	)


In [ ]:
# Comparisons Generator
def getComparisons(rate_later: list[VData], low_comparisons: list[VData], high_comparisons:list[VData], DCC: DistComparisonChecker):
	# Phase 0
	vid = None
	while True:
		(vid, cmp) = phase_init(rate_later, low_comparisons, DCC)
		yield cmp
		if not IGNORE_LIST.isIgnored(vid):
			break
		print('##### Current vid is ignored #####')

	while True:
		consolidated = phase_consolidate(high_comparisons, DCC)
		if consolidated:
			yield consolidated

		if get(vid, 0, 'individual_rating', 'n_comparisons') >= 2:
			(newvid, cmp) = phase_expand(vid, rate_later, DCC)
		else:
			(newvid, cmp) = phase_intricate(vid, low_comparisons, DCC)

		if not newvid:
			break
		yield cmp
		if not IGNORE_LIST.isIgnored(newvid):
			vid = newvid
		else:
			print('##### Current vid is ignored #####')

	return 'NO MORE'


# Main part

There are 3 cells bellow:

- "INIT/RESET": To run once to initialize the tool
- "IGNORE": To add videos to not be compared
- "CONTINUE": To run as many time as you want, to get more comparisons links generated

If at anytime you do comparisons not suggested by the tool, plase run again INIT/RESET to synchronize the tool with your current tournesol account

In [ ]:
# INIT/RESET ORDO (Replay this cell everytime any comparison other than suggested by this notebook has been made)
(RATE_LATER, LOW_CMPS, HIGH_CMPS) = get_lists()
comparison:str = getComparisons(RATE_LATER, LOW_CMPS, HIGH_CMPS, DCC)
comparisons_histo:list[str] = []
print('Initialized !\n')
print('[+]:', len(RATE_LATER))
print('[x]:', len(LOW_CMPS))
print('[*]:', len(HIGH_CMPS))

In [ ]:
# IGNORE: Add here videos to ignore if a suggestion told to compare a video you don't want to compare for now.
# You wan add more videos between ORDO steps, then continue ordo without RESET

# IGNORE_LIST.ignoreChannel('channelName')

# IGNORE_LIST.ignoreVideo('yt:abcdefghijk')

before = len(RATE_LATER) + len(LOW_CMPS) + len(HIGH_CMPS)
IGNORE_LIST.removeIgnored(RATE_LATER)
IGNORE_LIST.removeIgnored(LOW_CMPS)
IGNORE_LIST.removeIgnored(HIGH_CMPS)
print(f"Removed {before - len(RATE_LATER) - len(LOW_CMPS) - len(HIGH_CMPS)} videos from lists")

# -------------------------------- #

# Will prefetch data of private videos from these channels to improve distance calculation acuracy
CHNLS_TO_INIT = {
	'Tout Simplement – Kurzgesagt', 'ScienceClic English'
}

In [ ]:
# CONTINUE ORDO (Replay this cell everytime to get next comparison)
# Print previous
if comparisons_histo:
	print('Previous:')
	for (i,cmp) in enumerate(comparisons_histo[-5:], start=max(1, len(comparisons_histo)-4)):
		print(f'{i:4d}.', cmp)

# Print new
print()
comparisons_histo.append(next(comparison))
print('\nNew comparison:')
print(f'{len(comparisons_histo):4d}.', comparisons_histo[-1])

if len(comparisons_histo) <= 1:
	print('\nPrefetching private videos...')
	# Prefetch
	vids_to_prefetch = list(
		{v['entity']['uid'] for v in RATE_LATER if get(v, None, 'entity', 'metadata', 'uploader') in CHNLS_TO_INIT and not get(v, True, 'individual_rating', 'is_public')} |
		{v['entity']['uid'] for v in LOW_CMPS if get(v, None, 'entity', 'metadata', 'uploader') in CHNLS_TO_INIT and not get(v, True, 'individual_rating', 'is_public')} |
		{v['entity']['uid'] for v in HIGH_CMPS if get(v, None, 'entity', 'metadata', 'uploader') in CHNLS_TO_INIT and not get(v, True, 'individual_rating', 'is_public')}
	)
	for vid in vids_to_prefetch:
		DCC._update_cache(vid)
	print('Prefetch finished.')

In [ ]:
# All tournesol videos having 1 or 2 contributors and positive score
def public2contrib():
	print('Extracting...')
	allRes:list = callTournesolMulti('polls/videos/recommendations/', 'exclude_compared_entities=false&unsafe=true', start=2000, end=30000) # &metadata%5Blanguage%5D=en
	allRes = [vdata for vdata in allRes if 
		    get(vdata, 0, 'collective_rating', 'n_contributors') in [2]
		and get(vdata, 0, 'collective_rating', 'n_comparisons') >= 3
		and get(vdata, 0, 'collective_rating', 'tournesol_score')/get(vdata, 0, 'collective_rating', 'n_contributors') >= 5
	]
	allRes.sort(key=lambda vdata: (
		round(get(vdata, 0, 'collective_rating', 'tournesol_score')),
		get(vdata, 0, 'collective_rating', 'n_comparisons')
	), reverse=True)
	authors:dict[str,list] = dict()
	for vdata in allRes:
		a = get(vdata, '???', 'entity', 'metadata', 'uploader')
		authors.setdefault(a, list()).append(vdata)

	print('# Tournesol missing contributors list')
	print()
	print('Here are extracted all the videos in English that may go publically recommended with only one more contributor on [Tournesol](https://tournesol.app).')
	print()
	print('Extraction date:', str(datetime.datetime.now())[:16])
	print()
	print('Found', len(allRes), 'videos by', len(authors), 'channels')
	print()

	years={get(vdata, '???', 'entity', 'metadata', 'publication_date')[0:4] for vdata in allRes}

	for y in sorted(years):
		print(f'\n## Year {y}')
		for vdata in allRes:
			if get(vdata, '???', 'entity', 'metadata', 'publication_date')[0:4] == y:
				tscore=get(vdata, 0, 'collective_rating', 'tournesol_score')
				tcmps=get(vdata, 0, 'collective_rating', 'n_comparisons')
				tctrs=get(vdata, 0, 'collective_rating', 'n_contributors')
				tchannel=get(vdata, '???', 'entity', 'metadata', 'uploader').replace('\\', '\\\\').replace('(', '\(').replace(')', '\)').replace('[', '\[').replace(']', '\]')
				tname=get(vdata, '???', 'entity', 'metadata', 'name').replace('\\', '\\\\').replace('(', '\(').replace(')', '\)').replace('[', '\[').replace(']', '\]')
				tdur=get(vdata, 0, 'entity', 'metadata', 'duration')
				tminutes=f"{int(tdur/60):2d}:{tdur%60:02d}" if tdur < 3600 else f"{int(tdur/3600)}:{int((tdur%3600)/60):02d}:{tdur%60:02d}"
				print(f"- {tscore:3.0f}🌻 ({tcmps} cmps / {tctrs} user{'s' if tctrs > 1 else ''}) \[{tminutes}\] {tchannel}: [{tname}](https://tournesol.app/entities/{vdata['entity']['uid']})")

public2contrib()

In [ ]:
for vdata in RATE_LATER:
	print(f"[{get(vdata, '????-??-??', 'entity', 'metadata', 'publication_date')[:10]}, {get(vdata, 0, 'collective_rating', 'tournesol_score'):4.1f}, {get(vdata, 0, 'collective_rating', 'n_comparisons'):3d}/{get(vdata, 0, 'collective_rating', 'n_contributors')}, {get(vdata, 0, 'individual_rating', 'n_comparisons')}/me] {get(vdata, 'unknown', 'entity', 'metadata', 'uploader')} : {get(vdata, 'unknown', 'entity', 'metadata', 'name')}")
# print('[x]:', len(LOW_CMPS))
# print('[*]:', len(ALL_COMPARED))